In [29]:
import sys
import os

In [30]:
wd = os.getcwd()
sys.path.append(f'{wd}/../modules')

In [31]:
root_input_folder    = f'{wd}/../input_files'
root_output_folder   = f'{wd}/../output_files'

# Functions

In [32]:
def write_lj_fluid_hybrid_interaction_file(args, input_folder, output_folder, label):
    # Unpack arguments
    BOXZ, Lx, Ly, Lz, Lx_h, Ly_h, Lz_h, Lx_f, Ly_f, Lz_f, xlo_reflect, xhi_reflect, epsilon, sigma, eta, system_temp, pressure, time_step, n_step_to_run, m_frame_dump, m_frame_restart = args

    # System-specific constants
    kappa = 1000  # Spring constant for wall-wall interactions, in epsilon*sigma^-2
    r0 = 1.1626   # Equilibrium distance for wall-wall interactions, in sigma


    script = f"""# LJ Fluid with Hollow Flow Region
units           lj
dimension       3
#boundary        f p p
boundary        p p p
atom_style      full/gc/HAdResS

# Variables
variable     label          index   {label}
variable     BOXZ           equal   {BOXZ}            # Box Z length
variable     Lx             equal   {Lx}              # Box x length
variable     Ly             equal   {Ly}              # Box y length
variable     Lz             equal   {Lz}              # Box z length
variable     half_Lx        equal   ${{Lx}}/2.0
variable     half_Ly        equal   ${{Ly}}/2.0
variable     half_Lz        equal   ${{Lz}}/2.0
variable     Lx_h           equal   {Lx_h}            # Hollow region x length
variable     Ly_h           equal   {Ly_h}            # Hollow region y length
variable     Lz_h           equal   {Lz_h}            # Hollow region z length
variable     half_Lx_h      equal   ${{Lx_h}}/2.0
variable     half_Ly_h      equal   ${{Ly_h}}/2.0
variable     half_Lz_h      equal   ${{Lz_h}}/2.0
variable     Lx_f           equal   {Lx_f}            # Flow region x length
variable     Ly_f           equal   {Ly_f}            # Flow region y length
variable     Lz_f           equal   {Lz_f}            # Flow region z length
variable     half_Lx_f      equal   ${{Lx_f}}/2.0
variable     half_Ly_f      equal   ${{Ly_f}}/2.0
variable     half_Lz_f      equal   ${{Lz_f}}/2.0
variable     epsilon        equal   {epsilon}         # Interaction energy scale
variable     sigma          equal   {sigma}           # Particle size
variable     eta            equal   {eta}             # Interaction strength parameter
variable     system_temp    equal   {system_temp}     # System temperature
variable     pressure       equal   {pressure}        # System pressure
variable     dt             equal   {time_step}       # Time step
variable     kappa          equal   {kappa}           # Spring constant for wall-wall harmonic interactions
variable     r0             equal   {r0}              # Equilibrium distance for wall-wall interactions

molecule     mol1 {input_folder}/atom_details/LJ_A.dat

neighbor 0.5 bin
neigh_modify every 1 delay 0 check yes

lattice         fcc 0.9
region          box block -${{half_Lx}} ${{half_Lx}} -${{half_Ly}} ${{half_Ly}} -${{BOXZ}} ${{BOXZ}} units lattice
create_box      2 box

# Regions for boundary and flow
region          boundary_reg block  -${{half_Lx}} ${{half_Lx}}      -${{half_Ly}} ${{half_Ly}}      -${{half_Lz}} ${{half_Lz}}      units lattice
region          hollow block        -${{half_Lx_h}} ${{half_Lx_h}}  -${{half_Ly_h}} ${{half_Ly_h}}  -${{half_Lz_h}} ${{half_Lz_h}}  units lattice
region          flow_reg block      -${{half_Lx_f}} ${{half_Lx_f}}  -${{half_Ly_f}} ${{half_Ly_f}}  -${{half_Lz_f}} ${{half_Lz_f}}  units lattice

# Creating boundary atoms
create_atoms    2 region boundary_reg
group           boundary region boundary_reg 

# Creating hollow space in the boundary
delete_atoms    region hollow

# Creating flow atoms using molecule mol1
create_atoms    0 region flow_reg mol mol1 2851
group           flow region flow_reg

mass            1 1.0
mass            2 1.0

# Pair styles for non-bonded interactions

# Pair styles for non-bonded interactions
pair_style      hybrid/overlay lj/cut {2.5*sigma}
pair_modify     shift yes tail no mix geometric

pair_coeff   1 1 lj/cut {epsilon} {sigma} {2.5*sigma}          # s-s interactions
pair_coeff   1 2 lj/cut {epsilon} {sigma} {pow(2, 1/6)*sigma}  # s-w interactions using WCA potential
pair_coeff   2 2 lj/cut 0 0 {pow(2, 1/6)*sigma}                # w-w interactions
#pair_coeff  2 2 harmonic/cut {kappa} {r0} {2.5*sigma}         # w-w 

# Initialize velocities
reset_atom_ids
velocity      flow create {2*system_temp} 12345 dist uniform

#fix        1 flow nvt temp 2.0 2.0 10
fix         1 all npt temp {system_temp} {system_temp} 10 aniso {pressure} {pressure} 100

# Output settings
dump            trj all custom {m_frame_dump} ${{label}}.lammpstrj id type element x y z
thermo          20

compute myTemp flow temp
compute myPress all pressure myTemp
thermo_style    custom step temp etotal c_myTemp c_myPress

timestep        {time_step}

# Freeze wall particles
fix freezeWall boundary setforce 0.0 0.0 0.0 
#fix         wallBottom  all wall/reflect xlo {xlo_reflect}#-15
#fix         wallTop     all wall/reflect xhi {xhi_reflect}#14.6

# Continue with production run
run          {n_step_to_run}

write_restart {output_folder}/${{label}}.restart
"""

    # Write to file
    with open(f'{output_folder}/{label}.in', 'w') as file:
        file.write(script)

# Define experiment

In [33]:
import numpy as np
import pandas as pd
from spartian_tools import *

In [34]:
stage   = 'BUILD'
label   = 'bld_geometry'

BOXZ = 20

length_x = 30.0
length_y = 14.0
length_z = 10.0

length_hole_x = 30.0
length_hole_y = 14.0
length_hole_z = 8.8

length_flow_x = 30.0
length_flow_y = 14.0
length_flow_z = 7.0

xlo_reflect = -15
xhi_reflect = 14.6

time_step   = 0.001
system_temp = 1.57892098513 # Seems that the frozen particles affect the average, this is a correction

n_step_to_run   = 100000
m_frame_dump    = 300
m_frame_restart = 20000

pressure    = 2.65
epsilon     = 1
sigma       = 1
eta         = 1

In [35]:
experiment_label    = f'X{length_x}_YZ{length_y}_{length_z}_PRESS{pressure}_TEMP{system_temp}'
output_folder       = f'{root_output_folder}/{stage}/{experiment_label}'

create_experiment_folder(output_folder)

args = tuple([BOXZ, length_x, length_y, length_z, length_hole_x, length_hole_y, length_hole_z, length_flow_x, length_flow_y, length_flow_z, xlo_reflect, xhi_reflect, epsilon, sigma, eta, system_temp, pressure, time_step, n_step_to_run, m_frame_dump, m_frame_restart ])
write_lj_fluid_hybrid_interaction_file(args, root_input_folder, output_folder, label)

submit_file = f'{output_folder}/{label}.submit'
write_submit_jobs_SLURM(label, submit_file)
submit_jobs_SLURM(output_folder, submit_file)

Submitted batch job 67324
